<a href="https://colab.research.google.com/github/pushna99/CS682-Data-Center-NO2/blob/main/CS682_Project_Pushna_Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy import distance

from google.colab import drive
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
# Get the contents of NASA Timeseries files through URL
!wget https://so2.gsfc.nasa.gov/no2/pix/time_series/OMNO2_Timeseries_AllCities.csv -q
data=pd.read_csv('OMNO2_Timeseries_AllCities.csv',index_col=False, on_bad_lines='skip',skiprows=7)
df = pd.DataFrame(data)

In [ ]:
sum_med = df.groupby('City')['Median'].mean()
print(sum_med)

with open('OMNO2_Timeseries_AllCities.csv','r') as f_in:
    f = csv.reader(f_in)
    rows=df.groupby(['City','Country','LonCenter','LatCenter']).agg({'Median': ['mean']})
    rows.columns = ['Average NO2']
    rows = rows.reset_index()

geolocator = Nominatim(user_agent='myapplication')
for index, row in rows.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    rows.loc[index,"LatCenter"]=location.latitude
    rows.loc[index,"LonCenter"]=location.longitude
#Authorize access to google drive
drive.mount('/content/drive', force_remount=True)
try:
  nasa_cities_worksheet = pd.read_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/nasa-cities.xlsx')
  rows.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/nasa-cities.xlsx',index=False)
except FileNotFoundError:
  rows.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/nasa-cities.xlsx',index=False)

nasa_cities = rows.rename(columns={'City': 'City Name','LonCenter':'Longitude','LatCenter':'Latitude'})

City
Aberdeen     -0.241590
Abidjan      -0.289164
AbuDhabi      2.574397
Abuja         1.005829
Accra        -1.289991
               ...    
Yangoon     -13.736218
Yaounde      -0.751641
Yerevan       1.201822
Zhengzhou     6.617612
Zurich        1.628643
Name: Median, Length: 316, dtype: float64
Mounted at /content/drive


In [ ]:
#Read contents of lat-lon file
dfll = pd.read_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/lat-lon.xlsx')

#read contents of input file containing customer cities
dfcity = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/input.xlsx')

#Add two new columns "Latitude" and "Longitude"
dfcity["Latitude"] = pd.NaT
dfcity["Longitude"] = pd.NaT
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfcity.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    dfcity.loc[index,"Latitude"]=location.latitude
    dfcity.loc[index,"Longitude"]=location.longitude

#Drop NaT values and update the lat-lon file
dfcity=dfcity.dropna()
dfcity.update([dfcity.columns.values.tolist()] + dfcity.values.tolist())

customer_cities = pd.concat([dfcity,dfll], axis = 0).drop_duplicates().reset_index(drop=True)

In [ ]:
nasa_cities = nasa_cities.apply(pd.to_numeric, errors='ignore')
customer_cities = customer_cities.apply(pd.to_numeric, errors='ignore')
#NO2data = pd.merge(customer_cities,nasa_cities , on=['Longitude','Latitude'])
customer_cities["Average NO2"] = pd.NaT

# Check the nearest city based on latitude and longitude values
for index, row in customer_cities.iterrows():
  min_distance=float('inf')
  for nindex, nrow in nasa_cities.iterrows():
    if(distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles < min_distance):
      min_distance = distance.distance((row['Latitude'],row['Longitude']), (nrow['Latitude'],nrow['Longitude'])).miles
      min_index = nindex
  customer_cities.loc[index,'Average NO2'] = nasa_cities['Average NO2'].values[min_index]

In [ ]:
#Creating a new Average NO2 data in the output folder
customer_cities.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/average_no2_data.xlsx',index=False)